In [2]:
import pandas as pd
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [102]:
data = pd.read_csv('OPERATE_HIS.csv')

In [50]:
days = {2:[i for i in range(1,29)],3:[i for i in range(1,32)]}

### 利用率公式：$0.5\frac{载客时间}{载客时间+空载时间}+0.5\frac{载客里程}{载客里程+空载里程}$

#### 数据预处理

In [103]:
data['year_1'] = pd.Series(map(lambda x:x[-13:-9].rstrip('/'),data['EMPTY_BEGIN_TIME']))
data['year_2'] = pd.Series(map(lambda x:x[-13:-9].rstrip('/'),data['WORK_BEGIN_TIME']))
data['year_3'] = pd.Series(map(lambda x:x[-13:-9].rstrip('/'),data['WORK_END_TIME']))
data = data[data['year_1'] == '2017']
data = data[data['year_2'] == '2017']
data = data[data['year_3'] == '2017']
data.index = range(data.shape[0])
data['DAY'] = pd.Series(map(lambda x:x[:4].rstrip('/'),data['WORK_END_TIME']))
data['TIME'] = pd.Series(map(lambda x:int(x[-8:-6]),data['WORK_END_TIME']))

In [107]:
Utilization_rate = {}
for mouth in [2,3]:
    for day in tqdm(days[mouth]):
        pre_dataset = data[data['DAY'] == '{}/{}'.format(day,mouth)]
        pre_dataset.index = range(pre_dataset.shape[0])
        period = []
        for i in range(4):
            dataset = pre_dataset[pd.Series(map(lambda x:x >= i * 6 and x < (i+1) * 6,pre_dataset['TIME']))]
            dataset.index = range(dataset.shape[0])
            dataset['EMPTY_BEGIN_TIME'] = pd.to_datetime(dataset['EMPTY_BEGIN_TIME'])
            dataset['WORK_BEGIN_TIME'] = pd.to_datetime(dataset['WORK_BEGIN_TIME'])
            dataset['WORK_END_TIME'] = pd.to_datetime(dataset['WORK_END_TIME'])
            empty_time = pd.Series(map(lambda x:x.seconds,dataset['WORK_BEGIN_TIME']-dataset['EMPTY_BEGIN_TIME']))
            load_time = pd.Series(map(lambda x:x.seconds,dataset['WORK_END_TIME']-dataset['WORK_BEGIN_TIME']))
            time = load_time / (empty_time + load_time)
            mile = (dataset['LOAD_MILE'] / (dataset['LOAD_MILE'] + dataset['EMPTY_MILE'])).fillna(0)
            period.append(0.5*(time + mile).mean())
        Utilization_rate['{}/{}'.format(day,mouth)] = {
            'early_morning' : period[0],
            'morning' : period[1],
            'afternoon' : period[2],
            'night' : period[3]
        }

100%|██████████████████████████████████████████████████████████████████████████████████| 31/31 [24:24<00:00, 47.25s/it]


In [109]:
import json

In [110]:
with open('utilization.json','w') as f:
    json.dump(Utilization_rate,f)